# Plant Health AI Assistant - メモリ付き Runtime
## 概要
このノートブックでは、AWS Bedrock AgentCore を使用して、永続的なメモリ機能を備えた LangGraph ベースの植物健康解析システムをデプロイする方法を実演します。このシステムはマルチエージェントオーケストレーションとメモリストレージを組み合わせて、インテリジェントな植物診断と履歴追跡を提供します。

# 主要コンポーネント
1. マルチエージェント LangGraph ワークフロー
Entry Router: クエリが解析用か履歴取得用かを判断

Plant Detection Agent: 植物の種類と健康問題を特定

Care Agent: 専門的な治療アドバイスを提供

Web Search Agent: 最新の調査と推奨を検索

Memory Agents: 植物解析履歴の保存と取得

2. AWS Bedrock AgentCore Memory
植物解析セッションの永続ストレージ

アクターベースのメモリ分離（農家ごと）

植物健康追跡のための30日間の保持

会話形式のメモリフォーマット

3. Docker デプロイメント
AWS 最適化のための ARM64 コンテナ

AgentCore Runtime によるサーバーレススケーリング

監視とエラーハンドリングを備えた本番対応

前提条件
Bedrock AgentCore アクセス権を持つ AWS アカウント

MCP Gateway セットアップの完了（前のノートブックから）

Docker と AWS CLI の設定済み

Python 3.10+ 環境

# 01. 依存関係のインストールとクライアントの設定

In [ ]:
!pip install -q -r requirements.txt --no-cache-dir

print("✅ 依存関係がインストールされました。以下のセルを実行してください。")

In [ ]:
# カーネルを再起動
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json
import sys
import os
import time
import logging
from datetime import datetime
from typing import List, TypedDict, Any, Annotated
import base64
import random
import string
import shutil
import uuid
from PIL import Image
import io
import requests


from utils.utils import create_agentcore_role, create_agentcore_mem_role

from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
 
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.notebook import Runtime

# ノートブックの先頭に追加
import nest_asyncio
nest_asyncio.apply()


# コアクライアントと変数
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
region = boto3.session.Session().region_name
suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))

iam_client = boto3.client('iam')

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

## ノートブック 01 の設定を読み込み

In [ ]:
try:
    with open('plant_gateway_config.json', 'r') as f:
        config = json.load(f)
        
    REGION = config['REGION']
    GATEWAY_ID = config['GATEWAY_ID']
    GATEWAY_URL = config['GATEWAY_URL']
    COGNITO_INFO = config['COGNITO_INFO']
        
    print(f"ゲートウェイID: {GATEWAY_ID}")
    print(f"ゲートウェイURL: {GATEWAY_URL}")
    print(f"リージョン: {REGION}")
    print("✅ 設定を正常に読み込みました！")
    
except FileNotFoundError:
    print("❌ 設定ファイルが見つかりません。先にセットアップノートブックを実行してください！")

In [ ]:
# Gateway クライアントのセットアップ
gateway_client = GatewayClient(region_name=REGION)

# Memory クライアントのセットアップ
memory_client = MemoryClient(region_name=REGION)

# アクセストークンを取得
access_token = gateway_client.get_access_token_for_cognito(COGNITO_INFO)

# AgentCore Runtime と AgentCore Memory ロールの作成

In [ ]:
agent_name = "plant-advisor-agent-langgraph"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

print(f"ランタイムエージェントロール: {agentcore_iam_role}")
print("✅ ランタイムエージェントロールを正常に作成しました！")

agentcore_mem_name = "plant-advisor-mem-langgraph"
MEMORY_ROLE_ARN = create_agentcore_mem_role(agentcore_mem_name)

print(f"メモリエージェントロール: {agentcore_iam_role}")
print("✅ メモリエージェントロールを正常に作成しました！")

# 再利用のために Memory ロールと名前で設定を更新

In [ ]:
# タイムスタンプ付きの一意のメモリ名を作成

unique_memory_name = f"PlantHealthAdvisor_{timestamp}"

# 既存の設定を読み込み
try:
    with open('plant_gateway_config.json', 'r') as f:
        config = json.load(f)
    
    # メモリロール ARN を追加
    config['MEMORY_ROLE_ARN'] = MEMORY_ROLE_ARN
    config['MEMORY_NAME'] = unique_memory_name
    
    # 更新した設定を書き込み
    with open('plant_gateway_config.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"✅ 設定に MEMORY_ROLE_ARN を追加しました: {MEMORY_ROLE_ARN}")
    print(f"   MEMORY_NAME: {unique_memory_name}")
except Exception as e:
    print(f"❌ 設定の更新に失敗しました: {e}")

# Memory Store の作成

In [ ]:
memory_id = None

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

ACTOR_ID = "user_123"
SESSION_ID = "plant_analysis_session_001"  # 継続性のために同じセッションを再利用

# Memory Store を作成
try:
    print("メモリを作成中...")
    memory = memory_client.create_memory_and_wait(
        name=unique_memory_name,
        description="Plant Health Analysis Memory",
        strategies=[],  # 短期メモリにはストラテジーなし
        event_expiry_days=30,  # 植物解析を30日間保持
        memory_execution_role_arn=MEMORY_ROLE_ARN,
        max_wait=300,
        poll_interval=10
    )
    
    memory_id = memory['memoryId']

    # メモリ ID を追加
    config['MEMORY_ID'] = memory_id
    
    # 更新した設定を書き込み
    with open('plant_gateway_config.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"✅ メモリを正常に作成しました。ID: {memory_id}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # メモリが存在する場合、その ID を取得
        memories = memory_client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        print(f"✅ メモリは既に存在します。既存のメモリ ID を使用: {memory_id}")
    else:
        print(f"❌ メモリ作成エラー: {e}")
        
except Exception as e:
    print(f"❌ エラー: {e}")

# Langgraph 定義と Agent Runtime の準備

In [ ]:
# AgentCore App の初期化
app = BedrockAgentCoreApp()

COGNITO_DOMAIN=config['COGNITO_INFO']['token_endpoint']
COGNITO_CLIENT_ID=config['COGNITO_INFO']['client_id']
COGNITO_CLIENT_SECRET=config['COGNITO_INFO']['client_secret']

print("COGNITO_DOMAIN :  ", COGNITO_DOMAIN)
print("COGNITO_CLIENT_ID :  ", COGNITO_CLIENT_ID)
print("COGNITO_CLIENT_SECRET :  ", COGNITO_CLIENT_SECRET)

In [ ]:
# トークン生成のテスト
test_token = gateway_client.get_access_token_for_cognito(config['COGNITO_INFO'])
print(f"✅ トークンテスト: {test_token[:20]}..." if test_token else "❌ トークン取得に失敗しました")

In [ ]:
def parse_mcp_response(result):
    """デバッグ付きでネストされた MCP レスポンスをパース"""
    try:
        # 形式1: 直接の Lambda レスポンス
        if isinstance(result, dict) and 'plant_type' in result:
            return {
                'plant_name': result.get('plant_type', 'unknown'),
                'health_issues': result.get('health_analysis', '')
            }
        
        # 形式2: MCP でラップされたレスポンス
        if 'result' in result and 'content' in result['result']:
            content = result['result']['content'][0]['text']
            outer_json = json.loads(content)
            body_json = json.loads(outer_json['body'])
            
            # 植物検出レスポンス
            if 'plant_name' in body_json and 'health_issues' in body_json:
                print(f"✅ 植物をパースしました: {body_json.get('plant_name', 'unknown')}")
                return {
                    'plant_name': body_json.get('plant_name', 'unknown'),
                    'health_issues': body_json.get('health_issues', '')
                }
            
            # Web 検索レスポンス
            if 'web_search_results' in body_json:
                print(f"✅ Web 検索結果をパースしました")
                return {
                    'web_search_results': body_json.get('web_search_results', ''),
                    'plant_name': body_json.get('plant_name', 'unknown')
                }
            
            # 植物ケアレスポンス
            if 'expert_advice' in body_json:
                print(f"✅ 専門家のアドバイスをパースしました")
                return {
                    'expert_advice': body_json.get('expert_advice', ''),
                    'plant_name': body_json.get('plant_name', 'unknown')
                }
        
        print(f"❌ レスポンス形式をパースできませんでした")
        return None
        
    except Exception as e:
        print(f"❌ パースエラー: {e}")
        return None

In [ ]:
class PlantAnalysisState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    prompt: str           # ユーザークエリを追加
    query_type: str       # クエリタイプを追加: "analysis" または "history"
    image_path: str
    image_data: str
    plant_detection: dict
    health_issues: str
    expert_advice: str
    web_search_results: str
    final_report: str
    memory_status: str    # メモリ操作ステータスを追加

print("✅ メモリフィールドを含むステート定義を更新しました")

In [ ]:
def call_mcp_tool(tool_name: str, arguments: dict, bearer_token: str):
    """汎用 MCP ツール呼び出し関数"""
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {bearer_token}'}
    payload = {
        "jsonrpc": "2.0",
        "id": 3,
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments}
    }
    response = requests.post(GATEWAY_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
def create_plant_workflow():
    """3つの MCP ツール + メモリを使用した植物解析ワークフローを作成"""
    workflow = StateGraph(PlantAnalysisState)
    
    # 既存の関数をすべてそのまま維持
    def detect_plant(state: PlantAnalysisState) -> dict:
        """MCP Gateway ツールを使用した植物検出"""
        bearer_token = gateway_client.get_access_token_for_cognito(COGNITO_INFO)
        image_path = state.get('image_path', '')
        image_data = state.get('image_data', '')
        print(f"🔍 植物検出を開始: {image_path}")

        try: 
            if image_data:
                print(f"✅ 提供された image_data を使用: {len(image_data)} 文字")
                result = call_mcp_tool("plant-detection-target___plant_detection_tool", {
                    "image_data": image_data
                }, bearer_token)
            elif image_path and not image_path.startswith('s3://') and image_path != "from_image_data":
                if not os.path.exists(image_path):
                    print(f"❌ 画像ファイルが見つかりません: {image_path}")
                    return {"plant_detection": {"plant_type": "error"}, "health_issues": "画像ファイルが見つかりません"}

                with open(image_path, 'rb') as f:
                    image_bytes = f.read()
                    image_data_encoded = base64.b64encode(image_bytes).decode('utf-8')
                    print(f"✅ 画像をエンコードしました: {len(image_data_encoded)} 文字")

                result = call_mcp_tool("plant-detection-target___plant_detection_tool", {
                    "image_data": image_data_encoded
                }, bearer_token)
            else:
                result = call_mcp_tool("plant-detection-target___plant_detection_tool", {
                    "image_path": image_path
                }, bearer_token)

            parsed_result = parse_mcp_response(result)

            if parsed_result:
                plant_name = parsed_result.get("plant_name", "unknown")
                health_issues = parsed_result.get("health_issues", "")
                print(f"✅ 植物を検出: {plant_name}")
                print(f"📋 健康状態: {health_issues}")

                return {
                    "plant_detection": {"plant_type": plant_name},
                    "health_issues": health_issues
                }
        except Exception as e:
            print(f"❌ 植物検出エラー: {e}")

        return {"plant_detection": {"plant_type": "error"}, "health_issues": "検出に失敗しました"}

    def entry_router(state: PlantAnalysisState) -> dict:
        """クエリタイプに基づいてルーティング - 履歴または植物解析"""
        prompt = state.get("prompt", "")
        image_data = state.get("image_data", "")
        image_path = state.get("image_path", "")
        
        # 履歴キーワードをチェック
        if prompt:
            history_keywords = ["show me", "previous", "compare", "history", "analyses", "month", "last time"]
            if any(keyword in prompt.lower() for keyword in history_keywords):
                return {"next": "retrieve_memory"}
        
        # 画像データがある場合、植物検出へ
        if image_data or image_path:
            return {"next": "detect_plant"}
        
        return {"next": "END"}

    def analysis_router(state: PlantAnalysisState) -> str:
        """植物検出結果に基づいてルーティング - 元のロジック"""
        plant_detection = state.get("plant_detection", {})
        plant_name = plant_detection.get("plant_type", "").lower()
        health_issues = state.get("health_issues", "").lower()
        
        if not plant_name or plant_name == "error":
            return "END"
        
        # 重大な問題をチェック
        critical_keywords = ["severe", "dying", "critical", "emergency"]
        if any(keyword in health_issues for keyword in critical_keywords):
            return "urgent_consultation"
        
        return "expert_consultation"

    # 既存の関数をすべてそのまま維持
    def plant_care_agent(state: PlantAnalysisState) -> dict:
        """MCP Gateway ツールを使用した植物ケア"""
        bearer_token = gateway_client.get_access_token_for_cognito(COGNITO_INFO)
        plant_info = state.get('plant_detection', {})
        health_status = state.get('health_issues', '')
        plant_name = plant_info.get('plant_type', 'unknown plant')
        
        print(f"🌱 ケアアドバイスを取得中: {plant_name}")
        print(f"📋 健康状態: {health_status}")
        
        try:
            result = call_mcp_tool("plant-care-target___plant_care_tool", {
                "plant_name": plant_name,
                "health_status": health_status
            }, bearer_token)
            
            parsed_result = parse_mcp_response(result)
            
            if parsed_result and 'expert_advice' in parsed_result:
                advice = parsed_result['expert_advice']
                print(f"✅ ケアアドバイスを受信 ({len(advice)} 文字)")
                return {'expert_advice': advice}
                
        except Exception as e:
            print(f"❌ 植物ケアエラー: {e}")
        
        return {'expert_advice': '植物ケアアドバイスは利用できません'}

    def web_search_agent(state: PlantAnalysisState) -> dict:
        """MCP Gateway ツールを使用した Web 検索"""
        bearer_token = gateway_client.get_access_token_for_cognito(COGNITO_INFO)
        plant_info = state.get('plant_detection', {})
        health_status = state.get('health_issues', '')
        plant_name = plant_info.get('plant_type', 'unknown plant')
        
        print(f"🔍 Web 検索中: {plant_name}")
        
        try:
            result = call_mcp_tool("plant-web-search-target___plant_web_search_tool", {
                "plant_name": plant_name,
                "health_status": health_status
            }, bearer_token)
            
            parsed_result = parse_mcp_response(result)
            
            if parsed_result:
                search_results = parsed_result.get('web_search_results', str(parsed_result))
                print(f"✅ Web 検索完了 ({len(search_results)} 文字)")
                return {'web_search_results': search_results}
                
        except Exception as e:
            print(f"❌ Web 検索エラー: {e}")
        
        return {'web_search_results': 'Web 検索は利用できません'}

    def expert_consultation_agent(state: PlantAnalysisState) -> dict:
        """Web 検索へのフォールバック付き専門家相談"""
        care_result = plant_care_agent(state)
        
        if care_result.get('expert_advice') and 'unavailable' not in care_result.get('expert_advice', ''):
            return care_result
        
        web_result = web_search_agent(state)
        return {
            'expert_advice': web_result.get('web_search_results', 'アドバイスがありません'),
            'web_search_results': web_result.get('web_search_results', '')
        }

    def urgent_consultation_agent(state: PlantAnalysisState) -> dict:
        """緊急相談 - ケアと Web 検索の両方を使用"""
        care_result = plant_care_agent(state)
        web_result = web_search_agent(state)
        
        expert_advice = care_result.get('expert_advice', '')
        web_search_results = web_result.get('web_search_results', '')
        
        combined_advice = f"""**専門家のケアアドバイス:**
{expert_advice}

**追加の Web 調査:**
{web_search_results}"""
        
        return {
            'expert_advice': combined_advice,
            'web_search_results': web_search_results
        }
    
    def write_report(state: PlantAnalysisState) -> dict:
        """最終レポートを生成"""
        plant_info = state.get("plant_detection", {})
        health_issues = state.get("health_issues", "")
        expert_advice = state.get("expert_advice", "")
        web_search_results = state.get("web_search_results", "")
        
        report = f"""# 植物解析レポート

## 検出結果
- 植物の種類: {plant_info.get('plant_type', '不明')}
- 健康状態の評価: {health_issues}

## 専門家の推奨事項
{expert_advice}

## 解析方法
- ソース: MCP Gateway ツール
- 検出ツール: plant-detection-target___plant_detection_tool
- ケアツール: plant-care-target___plant_care_tool
- 検索ツール: plant-web-search-target___plant_web_search_tool
"""
        
        return {"final_report": report}
    
    # メモリ関数
    def retrieve_memory_agent(state: PlantAnalysisState) -> dict:
        """メモリから植物解析履歴を取得"""
        try:
            
            events = memory_client.list_events(
                memory_id=memory_id,
                actor_id=ACTOR_ID,
                session_id=SESSION_ID,
                max_results=10
            )
            
            if events:
                history_summary = "# 植物解析履歴\n\n"
                for i, event in enumerate(events, 1):
                    history_summary += f"{i}. {event}\n"
            else:
                history_summary = "# 植物解析履歴\n\n以前の解析が見つかりませんでした。"
            
            return {
                "final_report": history_summary,
                "memory_status": "retrieved"
            }
        except Exception as e:
            return {
                "final_report": f"# メモリエラー\n\n履歴を取得できませんでした: {str(e)}",
                "memory_status": "error"
            }

    def save_memory_agent(state: PlantAnalysisState) -> dict:
        """レポート生成後に植物解析をメモリに保存"""
        try:
            plant_info = state.get("plant_detection", {})
            health_issues = state.get("health_issues", "")
            expert_advice = state.get("expert_advice", "")
            
            conversation = [
                (f"Plant analysis for {plant_info.get('plant_type', 'unknown plant')}", "USER"),
                (f"Plant: {plant_info.get('plant_type')}\nHealth: {health_issues}\nAdvice: {expert_advice}", "ASSISTANT")
            ]
            
            memory_client.save_conversation(
                memory_id=memory_id,
                actor_id=ACTOR_ID,
                session_id=SESSION_ID,
                messages=conversation
            )
            
            current_report = state.get("final_report", "")
            enhanced_report = current_report + f"\n\n*解析が将来の参照用にメモリに保存されました*"
            
            return {
                "final_report": enhanced_report,
                "memory_status": "saved"
            }
        except Exception as e:
            current_report = state.get("final_report", "")
            enhanced_report = current_report + f"\n\n*メモリ保存に失敗しました: {str(e)}*"
            return {
                "final_report": enhanced_report,
                "memory_status": f"save_failed: {str(e)}"
            }
    
    # すべてのノードを追加
    workflow.add_node("entry_router", entry_router)              # エントリルーター
    workflow.add_node("detect_plant", detect_plant)
    workflow.add_node("urgent_consultation", urgent_consultation_agent)
    workflow.add_node("expert_consultation", expert_consultation_agent)
    workflow.add_node("write_report", write_report)
    workflow.add_node("retrieve_memory", retrieve_memory_agent)
    workflow.add_node("save_memory", save_memory_agent)
    
    # 2つのルーターでワークフローを構築
    workflow.set_entry_point("entry_router")  # エントリルーターから開始

    # 条件付きエッジで直接使用
    workflow.add_conditional_edges(
        "entry_router",
        lambda state: state["next"],  # ワークフローステートで "next" を検索
        {
            "detect_plant": "detect_plant",
            "retrieve_memory": "retrieve_memory",
            "END": END
        }
    )
    # 解析ルーターエッジ（植物検出後）
    workflow.add_conditional_edges(
        "detect_plant",
        analysis_router,  # 元のルーターロジック
        {
            "urgent_consultation": "urgent_consultation",
            "expert_consultation": "expert_consultation",
            "END": END
        }
    )
    
    # 解析完了パス
    workflow.add_edge("expert_consultation", "write_report")
    workflow.add_edge("urgent_consultation", "write_report")
    workflow.add_edge("write_report", "save_memory")
    workflow.add_edge("save_memory", END)
    
    # 履歴パス
    workflow.add_edge("retrieve_memory", END)
    
    return workflow.compile(checkpointer=MemorySaver())

In [ ]:
@app.entrypoint
def invoke(payload):
    """メモリサポート付き LangGraph MCP ワークフローの AgentCore エントリポイント"""
    # クリーンなログのために image_data を切り詰め
    log_payload = payload.copy()
    if 'image_data' in log_payload and log_payload['image_data']:
        log_payload['image_data'] = f"{log_payload['image_data'][:50]}... ({len(log_payload['image_data'])} chars)"
    
    print(f"📥 ペイロードを受信しました: {log_payload}")
    
    prompt = payload.get("prompt", "")
    image_path = payload.get("image_path", "")
    image_data = payload.get("image_data", "")
    
    # 履歴クエリかどうかをチェック
    history_keywords = ["show me", "previous", "compare", "history", "analyses", "month", "last time"]
    is_history_query = any(keyword in prompt.lower() for keyword in history_keywords) if prompt else False
    
    # 非履歴クエリの場合のみ画像が必要
    if not is_history_query and not image_path and not image_data:
        return {"error": "植物解析用の image_path または image_data が提供されていません", "status": "failed"}
    
    # すべての必須フィールドでステートを初期化
    initial_state = {
        "messages": [],
        "prompt": prompt,                                      # 追加: 履歴検出用
        "image_path": image_path or "from_image_data",
        "image_data": image_data,
        "plant_detection": {},
        "health_issues": "",
        "expert_advice": "",
        "web_search_results": "",
        "final_report": "",
        "memory_status": ""                                    # 追加: メモリ操作用
    }
    
    config = {"configurable": {"thread_id": f"agentcore_{random.randint(1000, 9999)}"}}
    
    try:
        final_state = langgraph_workflow.invoke(initial_state, config)
        
        return {
            "plant_type": final_state.get('plant_detection', {}).get('plant_type', 'Unknown'),
            "health_issues": final_state.get('health_issues', ''),
            "expert_advice": final_state.get('expert_advice', ''),
            "web_search_results": final_state.get('web_search_results', ''),
            "final_report": final_state.get('final_report', ''),
            "memory_status": final_state.get('memory_status', ''),    # 追加: メモリステータスを返す
            "status": "success"
        }
        
    except Exception as e:
        print(f"ワークフローエラー: {str(e)}")
        return {"error": str(e), "status": "failed"}

print("✅ メモリ付き LangGraph MCP ワークフローを作成しました")

# Langgraph のテスト

In [ ]:
image_path='./Image/sweet_potato_leaf.png'

try:
    print('🚀 植物エージェントをローカルでテスト中...')
    
    # 画像ファイルを読み込んでエンコード
    with open(image_path, 'rb') as f:
        image_bytes = f.read()
        base64_data = base64.b64encode(image_bytes).decode('utf-8')
        print(f"✅ 画像をエンコードしました: {len(base64_data)}文字")
    
    # テストペイロード
    test_payload = {
        'prompt': 'Analyze my plant',
        'image_data': base64_data
    }
    
    response = invoke(test_payload)
    print(f'✅ エージェント呼び出しが成功しました！')
    print(f'📝 レスポンスのコンテンツタイプ: {response.get("contentType")}')
    
    # レスポンスを処理 - 完全出力
    if response.get("contentType") == "application/json":
        content = []
        for chunk in response.get("response", []):
            #content.append(chunk.decode('utf-8'))
            content.append(chunk)
        result = json.loads(''.join(content))

        print(response)
        
        print(f"\n🌱 植物解析結果（完全版）:")
        print("=" * 80)
        
        print(f"\n📋 植物の種類: {result.get('plant_type', 'Unknown')}")
        
        print(f"\n🔍 健康状態の評価:")
        print(result.get('health_issues', 'None'))
        
        print(f"\n👨‍⚕️ 専門家のアドバイス:")
        print(result.get('expert_advice', 'No advice available'))
        
        if result.get('web_search_results'):
            print(f"\n🔍 Web検索結果:")
            print(result.get('web_search_results'))
        
        print(f"\n📄 最終レポート:")
        print(result.get('final_report', 'No report available'))
        
        print(f"\n✅ ステータス: {result.get('status', 'Unknown')}")
        
        print("=" * 80)
    else:
        print(f"Rawレスポンス: {response}")
    
    print('\n🎉 植物エージェントランタイムが実際の画像データで正常に動作しています！')

except Exception as e:
    print(f'❌ エージェント呼び出しに失敗しました: {e}')

# Runtime エージェントの準備

In [ ]:
# プロジェクトフォルダを作成
project_folder = "plant_agent_runtime"
os.makedirs(project_folder, exist_ok=True)

print(f"✅ フォルダを作成しました: {project_folder}")

# __init__.py ファイルを作成
init_content = '''"""
Plant Analysis Agent - メモリ付き LangGraph MCP ワークフロー
"""

__version__ = "1.0.0"
__author__ = "Plant Health Team"
'''

with open(f"{project_folder}/__init__.py", 'w') as f:
    f.write(init_content)

print("✅ __init__.py を作成しました")

# requirements.txt をコピー
files_to_copy = ['requirements.txt']

for file_name in files_to_copy:
    if os.path.exists(file_name):
        shutil.copy(file_name, f"{project_folder}/{file_name}")
        print(f"✅ コピーしました: {file_name}")
    else:
        print(f"❌ 見つかりません: {file_name}")

In [ ]:
with open('plant_gateway_config.json', 'r') as f:
    config = json.load(f)

workflow_code = f'''
import os
import sys
import time
import logging
import boto3
import string
from datetime import datetime
import requests
import json
from typing import List, TypedDict, Any
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError
import random
import nest_asyncio
import base64

from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

nest_asyncio.apply()

app = BedrockAgentCoreApp()

# Fixed configuration - no quotes around dict
COGNITO_INFO = {config['COGNITO_INFO']}
REGION = '{config.get('REGION')}'
GATEWAY_ID = '{config['GATEWAY_ID']}'
GATEWAY_URL = '{config['GATEWAY_URL']}'
MEMORY_ID = '{memory_id}'
MEMORY_ROLE_ARN = '{config['MEMORY_ROLE_ARN']}'
   
client = GatewayClient(region_name=REGION)    
memory_client = MemoryClient(region_name=REGION)

ACTOR_ID = "user_123"
SESSION_ID = "plant_analysis_session_001"

class PlantAnalysisState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    prompt: str           
    query_type: str       
    image_path: str
    image_data: str
    plant_detection: dict
    health_issues: str
    expert_advice: str
    recommended_fertilizer: str
    web_search_results: str
    final_report: str
    memory_status: str    
    order_status: str
    live_session_url: str
    message: str

def call_mcp_tool(tool_name: str, arguments: dict, bearer_token: str):
    headers = {{'Content-Type': 'application/json', 'Authorization': f'Bearer {{bearer_token}}'}}
    payload = {{
        "jsonrpc": "2.0",
        "id": 3,
        "method": "tools/call",
        "params": {{"name": tool_name, "arguments": arguments}}
    }}
    response = requests.post(GATEWAY_URL, headers=headers, json=payload)
    return response.json()

def parse_mcp_response(result):
    try:
        # Format 1: Direct Lambda response
        if isinstance(result, dict) and 'plant_type' in result:
            return {{
                'plant_name': result.get('plant_type', 'unknown'),
                'health_issues': result.get('health_analysis', '')
            }}
        
        # Format 2: MCP wrapped response
        if 'result' in result and 'content' in result['result']:
            content = result['result']['content'][0]['text']
            outer_json = json.loads(content)
            body_json = json.loads(outer_json['body'])  # Double parsing needed
            
            # Plant detection response
            if 'plant_name' in body_json and 'health_issues' in body_json:
                return {{
                    'plant_name': body_json.get('plant_name', 'unknown'),
                    'health_issues': body_json.get('health_issues', '')
                }}
            
            # Web search response
            if 'web_search_results' in body_json:
                return {{
                    'web_search_results': body_json.get('web_search_results', ''),
                    'plant_name': body_json.get('plant_name', 'unknown')
                }}
            
            # Plant care response  
            if 'expert_advice' in body_json:
                return {{
                    'expert_advice': body_json.get('expert_advice', ''),
                    'plant_name': body_json.get('plant_name', 'unknown')
                }}
        
        return None
        
    except Exception as e:
        print(f"Parse error: {{str(e)}}")  # Fixed f-string
        return None


def create_plant_workflow():
    workflow = StateGraph(PlantAnalysisState)
    
    def detect_plant(state: PlantAnalysisState) -> dict:
        try:
            bearer_token = client.get_access_token_for_cognito(COGNITO_INFO)
        except Exception as token_error:
            raise token_error
        
        image_data = state.get('image_data', '')
        image_path = state.get('image_path', '')  # Fixed: Added missing variable

        try: 
            if image_data:
                print(f"✅ Using provided image_data: {{len(image_data)}} characters")
                result = call_mcp_tool("plant-detection-target___plant_detection_tool", {{
                    "image_data": image_data
                }}, bearer_token)
            elif image_path and not image_path.startswith('s3://') and image_path != "from_image_data":
                if not os.path.exists(image_path):
                    print(f"❌ Image file not found: {{image_path}}")
                    return {{"plant_detection": {{"plant_type": "error"}}, "health_issues": "Image file not found"}}

                with open(image_path, 'rb') as f:
                    image_bytes = f.read()
                    image_data_encoded = base64.b64encode(image_bytes).decode('utf-8')
                    print(f"✅ Image encoded: {{len(image_data_encoded)}} characters")

                result = call_mcp_tool("plant-detection-target___plant_detection_tool", {{
                    "image_data": image_data_encoded
                }}, bearer_token)
            else:
                result = call_mcp_tool("plant-detection-target___plant_detection_tool", {{
                    "image_path": image_path
                }}, bearer_token)

            parsed_result = parse_mcp_response(result)

            if parsed_result:
                plant_name = parsed_result.get("plant_name", "unknown")
                health_issues = parsed_result.get("health_issues", "")
                print(f"✅ Plant detected: {{plant_name}}")
                print(f"📋 Health status: {{health_issues}}")

                return {{
                    "plant_detection": {{"plant_type": plant_name}},
                    "health_issues": health_issues
                }}
        except Exception as e:
            print(f"❌ Plant detection error: {{e}}")

        return {{"plant_detection": {{"plant_type": "error"}}, "health_issues": "Detection failed"}}

    def entry_router(state: PlantAnalysisState) -> dict:
        prompt = state.get("prompt", "")
        image_data = state.get("image_data", "")
        image_path = state.get("image_path", "")
        
        if prompt:
            history_keywords = ["show me", "previous", "compare", "history", "analyses", "month", "last time"]
            if any(keyword in prompt.lower() for keyword in history_keywords):
                return {{"next": "retrieve_memory"}}
        
        if image_data or image_path:
            return {{"next": "detect_plant"}}
        
        return {{"next": "END"}}
        
    def analysis_router(state: PlantAnalysisState) -> str:
        plant_detection = state.get("plant_detection", {{}})
        plant_name = plant_detection.get("plant_type", "").lower()
        health_issues = state.get("health_issues", "").lower()
        
        if not plant_name or plant_name == "error":
            return "END"
        
        critical_keywords = ["severe", "dying", "critical", "emergency"]
        if any(keyword in health_issues for keyword in critical_keywords):
            return "urgent_consultation"
        
        return "expert_consultation"

    def plant_care_agent(state: PlantAnalysisState) -> dict:
        bearer_token = client.get_access_token_for_cognito(COGNITO_INFO)
        plant_info = state.get('plant_detection', {{}})
        health_status = state.get('health_issues', '')
        plant_name = plant_info.get('plant_type', 'unknown plant')
        
        print(f"🌱 Getting care advice for: {{plant_name}}")
        print(f"📋 Health status: {{health_status}}")
        
        try:
            result = call_mcp_tool("plant-care-target___plant_care_tool", {{
                "plant_name": plant_name,
                "health_status": health_status
            }}, bearer_token)
            
            parsed_result = parse_mcp_response(result)
            
            if parsed_result and 'expert_advice' in parsed_result:
                advice = parsed_result['expert_advice']
                print(f"✅ Care advice received ({{len(advice)}} chars)")
                fertilizer = parsed_result.get('recommended_fertilizer', 'All Purpose Plant Fertilizer')
                return {{'expert_advice': advice, 'recommended_fertilizer': fertilizer}}
                
        except Exception as e:
            print(f"❌ Plant care error: {{e}}")
        
        return {{'expert_advice': 'Plant care advice unavailable'}}

    def web_search_agent(state: PlantAnalysisState) -> dict:
        bearer_token = client.get_access_token_for_cognito(COGNITO_INFO)
        plant_info = state.get('plant_detection', {{}})
        health_status = state.get('health_issues', '')
        plant_name = plant_info.get('plant_type', 'unknown plant')
        
        print(f"🔍 Web searching for: {{plant_name}}")
        
        try:
            result = call_mcp_tool("plant-web-search-target___plant_web_search_tool", {{
                "plant_name": plant_name,
                "health_status": health_status
            }}, bearer_token)
            
            parsed_result = parse_mcp_response(result)
            
            if parsed_result:
                search_results = parsed_result.get('web_search_results', str(parsed_result))
                print(f"✅ Web search completed ({{len(search_results)}} chars)")
                return {{'web_search_results': search_results}}
                
        except Exception as e:
            print(f"❌ Web search error: {{e}}")
        
        return {{'web_search_results': 'Web search unavailable'}}

    def expert_consultation_agent(state: PlantAnalysisState) -> dict:
        care_result = plant_care_agent(state)
        
        if care_result.get('expert_advice') and 'unavailable' not in care_result.get('expert_advice', ''):
            return care_result
        
        web_result = web_search_agent(state)
        return {{
            'expert_advice': web_result.get('web_search_results', 'No advice available'),
            'web_search_results': web_result.get('web_search_results', '')
        }}

    def urgent_consultation_agent(state: PlantAnalysisState) -> dict:
        care_result = plant_care_agent(state)
        web_result = web_search_agent(state)
        
        expert_advice = care_result.get('expert_advice', '')
        web_search_results = web_result.get('web_search_results', '')
        
        combined_advice = f"""**Expert Care Advice:**
{{expert_advice}}

**Additional Web Research:**
{{web_search_results}}"""
        
        return {{
            'expert_advice': combined_advice,
            'web_search_results': web_search_results
        }}
    
    def write_report(state: PlantAnalysisState) -> dict:
        plant_info = state.get("plant_detection", {{}})
        health_issues = state.get("health_issues", "")
        expert_advice = state.get("expert_advice", "")
        
        report = f"""# Plant Analysis Report

## Detection Results
- Plant Type: {{plant_info.get('plant_type', 'Unknown')}}
- Health Assessment: {{health_issues}}

## Expert Recommendations
{{expert_advice}}
"""
        
        return {{"final_report": report}}
    
    def retrieve_memory_agent(state: PlantAnalysisState) -> dict:
        try:
            events = memory_client.list_events(
                memory_id=MEMORY_ID,
                actor_id=ACTOR_ID,
                session_id=SESSION_ID,
                max_results=10
            )
            
            if events:
                history_summary = "# Plant Analysis History\\n\\n"
                for i, event in enumerate(events, 1):
                    history_summary += f"{{i}}. {{event}}\\n"
            else:
                history_summary = "# Plant Analysis History\\n\\nNo previous analyses found."
            
            return {{
                "final_report": history_summary,
                "memory_status": "retrieved"
            }}
        except Exception as e:
            return {{
                "final_report": f"# Memory Error\\n\\nCould not retrieve history: {{str(e)}}",
                "memory_status": "error"
            }}

    
    def save_memory_agent(state: PlantAnalysisState) -> dict:
        try:
            plant_info = state.get("plant_detection", {{}})
            health_issues = state.get("health_issues", "")
            expert_advice = state.get("expert_advice", "")
            
            conversation = [
                (f"Plant analysis for {{plant_info.get('plant_type', 'unknown plant')}}", "USER"),
                (f"Plant: {{plant_info.get('plant_type')}}\\nHealth: {{health_issues}}\\nAdvice: {{expert_advice}}", "ASSISTANT")
            ]
            
            memory_client.save_conversation(
                memory_id=MEMORY_ID,
                actor_id=ACTOR_ID,
                session_id=SESSION_ID,
                messages=conversation
            )
            
            current_report = state.get("final_report", "")
            enhanced_report = current_report + "\\n\\n*Analysis saved to memory for future reference*"
            
            return {{
                "final_report": enhanced_report,
                "memory_status": "saved"
            }}
        except Exception as e:
            current_report = state.get("final_report", "")
            enhanced_report = current_report + f"\\n\\n*Memory save failed: {{str(e)}}*"
            return {{
                "final_report": enhanced_report,
                "memory_status": f"save_failed: {{str(e)}}"
            }}
    
    # Add all nodes
    workflow.add_node("entry_router", entry_router)
    workflow.add_node("detect_plant", detect_plant)
    workflow.add_node("urgent_consultation", urgent_consultation_agent)
    workflow.add_node("expert_consultation", expert_consultation_agent)
    workflow.add_node("write_report", write_report)
    workflow.add_node("retrieve_memory", retrieve_memory_agent)
    workflow.add_node("save_memory", save_memory_agent)
    
    workflow.set_entry_point("entry_router")

    workflow.add_conditional_edges(
        "entry_router",
        lambda state: state["next"],
        {{
            "detect_plant": "detect_plant",
            "retrieve_memory": "retrieve_memory",
            "END": END
        }}
    )
    
    workflow.add_conditional_edges(
        "detect_plant",
        analysis_router,
        {{
            "urgent_consultation": "urgent_consultation",
            "expert_consultation": "expert_consultation",
            "END": END
        }}
    )
    
    workflow.add_edge("expert_consultation", "write_report")
    workflow.add_edge("urgent_consultation", "write_report")
    workflow.add_edge("write_report", "save_memory")
    workflow.add_edge("save_memory", END)
    workflow.add_edge("retrieve_memory", END)
    
    return workflow.compile(checkpointer=MemorySaver())   

langgraph_workflow = create_plant_workflow()

@app.entrypoint
def invoke(payload):
    log_payload = payload.copy()
    if 'image_data' in log_payload and log_payload['image_data']:
        log_payload['image_data'] = f"{{log_payload['image_data'][:50]}}... ({{len(log_payload['image_data'])}} chars)"
    
    print(f"📥 Received payload: {{log_payload}}")
    
    prompt = payload.get("prompt", "")
    image_path = payload.get("image_path", "")
    image_data = payload.get("image_data", "")
    
    history_keywords = ["show me", "previous", "compare", "history", "analyses", "month", "last time"]
    is_history_query = any(keyword in prompt.lower() for keyword in history_keywords) if prompt else False
    
    if not is_history_query and not image_path and not image_data:
        return {{"error": "No image_path or image_data provided for plant analysis", "status": "failed"}}
    
    initial_state = {{
        "messages": [],
        "prompt": prompt,
        "query_type": "history" if is_history_query else "analysis",
        "image_path": image_path or "from_image_data",
        "image_data": image_data,
        "plant_detection": {{}},
        "health_issues": "",
        "expert_advice": "",
        "recommended_fertilizer": "",
        "web_search_results": "",
        "final_report": "",
        "memory_status": "",
        "order_status": "",
        "live_session_url": "",
        "message": ""
    }}
    
    config = {{"configurable": {{"thread_id": f"agentcore_{{random.randint(1000, 9999)}}"}}}}
    
    try:
        final_state = langgraph_workflow.invoke(initial_state, config)
        
        return {{
            "plant_type": final_state.get('plant_detection', {{}}).get('plant_type', 'Unknown'),
            "health_issues": final_state.get('health_issues', ''),
            "expert_advice": final_state.get('expert_advice', ''),
            "recommended_fertilizer": final_state.get('recommended_fertilizer', 'All Purpose Plant Fertilizer'),
            "web_search_results": final_state.get('web_search_results', ''),
            "final_report": final_state.get('final_report', ''),
            "memory_status": final_state.get('memory_status', ''),
            "status": "success"
        }}
        
    except Exception as e:
        print(f"Workflow error: {{str(e)}}")
        return {{"error": str(e), "status": "failed"}}

print("✅ Enhanced LangGraph MCP Workflow with Memory created")

if __name__ == "__main__":
    print("🚀 Starting Plant Analysis Agent...")
    print(f"🔗 Gateway: {{GATEWAY_ID}}")
    print(f"🌐 MCP URL: {{GATEWAY_URL}}")
    print(f"🧠 Memory ID: {{MEMORY_ID}}")
    print(f"👤 Actor ID: {{ACTOR_ID}}")
    print(f"📝 Session ID: {{SESSION_ID}}")
    app.run()
'''

with open(f'{project_folder}/plant_workflow_memory.py', 'w') as f:
    f.write(workflow_code)

print("✅ Workflow saved to 'plant_workflow_memory.py'")
print("📝 Remember to update the configuration values in the file!")


# Starter Toolkit を使用したデプロイ

In [ ]:
# # Dockerfile が存在するか確認しその内容を確認
if os.path.exists('Dockerfile'):
    with open('Dockerfile', 'r') as f:
        content = f.read()
    print(f"Dockerfile サイズ: {len(content)} バイト")
else:
    print("Dockerfile が見つかりません")

# 破損したファイルを削除
for file in ['Dockerfile', '.dockerignore', '.bedrock_agentcore.yaml']:
    if os.path.exists(file):
        os.remove(file)
        
# 解決策: プロンプト関数にモンキーパッチを適用して自動応答
import sys
from unittest.mock import patch

def mock_prompt(text, default=""):
    """プロンプトにデフォルト値で自動応答"""
    print(f"プロンプトに自動応答: {text}")
    if "memory" in text.lower():
        return "no"  # 長期メモリを無効化
    return default or "no"

# configure の前にプロンプト関数にパッチを適用
runtime = Runtime()
with patch('bedrock_agentcore_starter_toolkit.cli.common.prompt', side_effect=mock_prompt):
    config = runtime.configure(
        entrypoint=f'{project_folder}/plant_workflow_memory.py',
        requirements_file=f'{project_folder}/requirements.txt',
        agent_name="plant_advisor_agent",
        auto_create_ecr=True,
        execution_role=agentcore_iam_role['Role']['Arn']
    )

# Dockerfile が正しく作成されたことを確認
if os.path.exists('Dockerfile'):
    with open('Dockerfile', 'r') as f:
        content = f.read()
    print(f"新しい Dockerfile サイズ: {len(content)} バイト")
    if len(content) > 100:
        print("✅ Dockerfile が正常に生成されました")
        runtime.launch()
    else:
        print("❌ Dockerfile はまだ破損しています")
else:
    print("❌ Dockerfile が生成されていません")

# Runtime の呼び出し

In [ ]:
image_path='./Image/sweet_potato_leaf.png'

try:
    print('🚀 実際の画像で植物エージェントランタイムをテスト中...')
    
    # 画像ファイルを読み込んでエンコード
    with open(image_path, 'rb') as f:
        image_bytes = f.read()
        base64_data = base64.b64encode(image_bytes).decode('utf-8')
        print(f"✅ 画像をエンコードしました: {len(base64_data)}文字")
    
    # テストペイロード
    test_payload = {
        'prompt': 'Analyze my plant',
        'image_data': base64_data
    }
    
    response = runtime.invoke(payload=test_payload)
    print(f'✅ エージェント呼び出しが成功しました！')
    print(f'📝 レスポンスのコンテンツタイプ: {response.get("contentType")}')
    print(response)
    
    # レスポンスを処理 - 完全出力
    if response.get("contentType") == "application/json":
        content = []
        for chunk in response.get("response", []):
            #content.append(chunk.decode('utf-8'))
            content.append(chunk)
        result = json.loads(''.join(content))

        print(response)
        
        print(f"\n🌱 植物解析結果（完全版）:")
        print("=" * 80)
        
        print(f"\n📋 植物の種類: {result.get('plant_type', 'Unknown')}")
        
        print(f"\n🔍 健康状態の評価:")
        print(result.get('health_issues', 'None'))
        
        print(f"\n👨‍⚕️ 専門家のアドバイス:")
        print(result.get('expert_advice', 'No advice available'))
        
        if result.get('web_search_results'):
            print(f"\n🔍 Web検索結果:")
            print(result.get('web_search_results'))
        
        print(f"\n📄 最終レポート:")
        print(result.get('final_report', 'No report available'))
        
        print(f"\n✅ ステータス: {result.get('status', 'Unknown')}")
        
        print("=" * 80)
    else:
        print(f"Rawレスポンス: {response}")
    
    print('\n🎉 植物エージェントランタイムが実際の画像データで正常に動作しています！')

except Exception as e:
    print(f'❌ エージェント呼び出しに失敗しました: {e}')

In [ ]:
def extract_clean_plant_analysis(response_data):
    """クリーンで読みやすい植物解析を抽出"""
    
    response_bytes = response_data.get('response', [])
    full_response = b''.join(response_bytes).decode('utf-8')
    
    try:
        parsed_response = json.loads(full_response)
        final_report = parsed_response.get('final_report', '')
        
        # 植物解析セクションを抽出
        # "Plant: ... Health: ... Advice: ..." パターンを検索
        plant_analyses = []
        
        # 番号付きエントリで分割
        entries = re.split(r'\d+\.\s*{', final_report)
        
        for entry in entries[1:]:  # 最初の分割部分をスキップ
            # 会話データ内のテキストコンテンツを検索
            text_matches = re.findall(r"'text': '([^']*(?:\\.[^']*)*)'", entry)
            
            for text in text_matches:
                cleaned_text = text.replace('\\\\n', '\n').replace("\\'", "'")
                
                # 植物解析のように見えるかチェック
                if 'Plant:' in cleaned_text and 'Health:' in cleaned_text:
                    # セクションに分割
                    sections = cleaned_text.split('\n')
                    
                    plant_info = ""
                    health_info = ""
                    advice_info = ""
                    current_section = None
                    
                    for line in sections:
                        if line.startswith('Plant:'):
                            plant_info = line
                            current_section = 'plant'
                        elif line.startswith('Health:'):
                            health_info = line
                            current_section = 'health'
                        elif line.startswith('Advice:'):
                            advice_info = line
                            current_section = 'advice'
                        elif current_section == 'health' and line.strip():
                            health_info += "\n" + line
                        elif current_section == 'advice' and line.strip():
                            advice_info += "\n" + line
                    
                    if plant_info and health_info:
                        plant_analyses.append({
                            'plant': plant_info,
                            'health': health_info,
                            'advice': advice_info
                        })
        
        return plant_analyses
        
    except Exception as e:
        print(f"解析の抽出中にエラー: {e}")
        return []

In [ ]:
try:
    print('🚀 植物エージェントランタイムのメモリをテスト中...')
    
    # メモリのテスト
    test_payload = {
        'prompt': "Show me my previous plant analyses"
    }
    
    response = runtime.invoke(payload=test_payload)
    print(f'✅ エージェント呼び出しが成功しました！')
    print(f'📝 レスポンスのコンテンツタイプ: {response.get("contentType")}')

    # レスポンスを処理 - 完全出力
    if response.get("contentType") == "application/json":

        response_data = response['response']
        if isinstance(response_data, list) and len(response_data) > 0:
            if isinstance(response_data[0], str):
                # レスポンスが文字列の場合、直接結合
                full_response = ''.join(response_data)
            else:
                # レスポンスがバイトの場合、先にデコード
                full_response = b''.join(response_data).decode('utf-8')
        else:
            full_response = str(response_data)


        
        parsed = json.loads(full_response)

        final_report= parsed.get('final_report')
        
        print("=== 抽出されたコンテンツ ===")
        print(f"ステータス: {parsed.get('status')}")
        print(f"メモリステータス: {parsed.get('memory_status')}")
        print(f"\n最終レポート:\n{final_report}")

        
    print('\n🎉 植物エージェントランタイムがメモリで正常に動作しています！')

except Exception as e:
    print(f'❌ エージェント呼び出しに失敗しました: {e}')